In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout

import keras.backend as K


from typing import Optional

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [4]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI_AD/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 352 images belonging to 2 classes.


In [5]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI_AD/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 130 images belonging to 2 classes.


In [6]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


In [7]:
for layer in base_model.layers:
    layer.trainable=False


In [8]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [10]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) 

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [11]:

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [12]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [16]:
history=model.fit(train_dataset,
                        epochs = 30,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/30
11/11 [==============================] - 7s 612ms/step - loss: 0.4828 - accuracy: 0.7614 - precision: 0.7110 - recall: 0.8807 - auc: 0.8350 - f1_score: 0.7803 - lr: 0.0100
Epoch 2/30
11/11 [==============================] - 5s 470ms/step - loss: 0.4852 - accuracy: 0.7528 - precision: 0.7051 - recall: 0.8693 - auc: 0.8355 - f1_score: 0.7752 - lr: 0.0063
Epoch 3/30
11/11 [==============================] - 6s 573ms/step - loss: 0.4335 - accuracy: 0.8097 - precision: 0.7559 - recall: 0.9148 - auc: 0.8873 - f1_score: 0.8256 - lr: 0.0040
Epoch 4/30
11/11 [==============================] - 5s 467ms/step - loss: 0.4203 - accuracy: 0.8295 - precision: 0.7816 - recall: 0.9148 - auc: 0.8833 - f1_score: 0.8393 - lr: 0.0025
Epoch 5/30
11/11 [==============================] - 5s 472ms/step - loss: 0.3967 - accuracy: 0.8153 - precision: 0.7789 - recall: 0.8807 - auc: 0.8985 - f1_score: 0.8297 - lr: 0.0016
Epoch 6/30
11/11 [==============================] - 6s 577ms/step - loss: 0.4071 - ac

In [17]:
scores = model.evaluate(test_dataset)

5/5 [==============================] - 1s 136ms/step - loss: 0.3190 - accuracy: 0.8769 - precision: 0.8298 - recall: 1.0000 - auc: 0.9498 - f1_score: 0.9234


In [18]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.8769230842590332
Precision =  0.8297872543334961
Recall =  1.0
AUC =  0.9498274326324463
F1_score =  0.9233600497245789
